In [ ]:
] activate .

In [ ]:
push!(LOAD_PATH, joinpath(pwd(), "../../"))

## Differentiate through an ODE solver

In [ ]:
using OrdinaryDiffEq

function lotka_volterra(du,u,p,t)
  x, y = u
  α, β, δ, γ = p
  du[1] = dx = α*x - β*x*y
  du[2] = dy = -δ*y + γ*x*y
end

u0 = [1.0,1.0]
tspan = (0.0,10.0)
p = [1.5,1.0,3.0,1.0]
prob = ODEProblem(lotka_volterra,u0,tspan,p)
sol = solve(prob,Tsit5())

using Plots
plot(sol)

In [ ]:
using Flux, DiffEqFlux
p = [2.2, 1.0, 2.0, 0.4] # Initial Parameter Vector

function predict_adjoint() # Our 1-layer neural network
  Array(concrete_solve(prob,Tsit5(),u0,p,saveat=0.0:0.1:10.0))
end

Next we choose a loss function. Our goal will be to find parameter that make the Lotka-Volterra solution constant $x(t)=1$, so we defined our loss as the squared distance from 1:

In [ ]:
loss_adjoint() = sum(abs2,x-1 for x in predict_adjoint())

In [ ]:
using Flux: throttle
data = Iterators.repeated((), 100)
opt = ADAM(0.1)
cb = function () #callback function to observe training
    if (@isdefined IJulia)
        # "animation" in jupyter
        IJulia.clear_output(true)
    end
    display(loss_adjoint())
    # using `remake` to re-create our `prob` with current parameters `p`
    display(plot(solve(remake(prob,p=p),Tsit5(),saveat=0.0:0.1:10.0),ylim=(0,6)))
end

# Display the ODE with the initial parameter values.
cb()

Flux.train!(loss_adjoint, Flux.params(p), data, opt, cb = throttle(cb, .1))

Even cooler trebuchet example: https://fluxml.ai/2019/03/05/dp-vs-rl.html